## Training U-Net for Image Segmentation

In [ ]:
import utils

gt_info = utils.get_tiff_info(
    "/home/ubuntu/bdrs_ex_1/data/ground_truth/GBDA24_ex2_ref_data.tif"
)
gt_info

{'size': (21424, 13572),
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'gsd': (8.333333333333333e-05, 8.333333333333333e-05),
 'bounds': BoundingBox(left=21.331833333333332, bottom=39.12975, right=23.117166666666666, top=40.26075),
 'transform': Affine(8.333333333333333e-05, 0.0, 21.331833333333332,
        0.0, -8.333333333333333e-05, 40.26075),
 'band_count': 1,
 'tags': {'algorithm_version': 'V2.0.0',
  'copyright': 'ESA WorldCover project 2021 / Contains modified Copernicus Sentinel data (2021) processed by ESA WorldCover consortium',
  'creation_time': '2022-10-21 07:35:40.236136',
  'legend': '10  Tree cover\n20  Shrubland\n30  Grassland\n40  Cropland\n50  Built-up\n60  Bare/sparse vegetation\n70  Snow